In [1]:
import os
import re
import json
import random
import sys
import asyncio
import pickle
import datetime
import gurobipy as gp
sys.path.append('../')

from openai import OpenAI, AsyncClient
from json import JSONDecodeError
from enum import Enum
from tqdm.auto import tqdm
from utils import *
from pydantic import BaseModel
from colorama import Fore, Style
from llama_index.program.openai import OpenAIPydanticProgram
from llama_index.llms.openai import OpenAI

c:\Users\HAOXUAN\miniconda3\envs\or\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = json.load(open('../configs./configs.json', 'r'))
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
asyncclient = AsyncClient(api_key=os.environ["OPENAI_API_KEY"])

dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')

In [3]:
DATA_DIR = '../data'
DATASET_NAME = 'LPWP' 
OUTPUT_DIR = '../output'  

nl4opt_data = read_txt_file(os.path.join(DATA_DIR, DATASET_NAME, 'lpwp.txt'))
questions, answers = get_nl4opt_qas(nl4opt_data)
assert len(questions) == len(answers)

qa_pairs = list(zip(questions, answers))
# demo_samples, test_samples = get_demo_and_test_samples(qa_pairs)

questions = [q for q, _ in qa_pairs]
answers = [a for _, a in qa_pairs]

2024-09-19 00:09:53.910 | DEBUG    | utils:read_txt_file:15 - Reading file: ../data\LPWP\lpwp.txt
2024-09-19 00:09:53.912 | DEBUG    | utils:read_txt_file:17 - File read successfully: ../data\LPWP\lpwp.txt
2024-09-19 00:09:53.913 | INFO     | utils:get_nl4opt_qas:36 - Number of questions: 288
2024-09-19 00:09:53.913 | INFO     | utils:get_nl4opt_qas:37 - Number of answers: 288


In [4]:
class VarType(Enum):
    INTEGER = "INTEGER"
    CONTINUOUS = "CONTINUOUS"

class CodeScheme(BaseModel):
    """The variable type in the problem and the code string"""
    
    var_type: VarType
    code: str     

In [92]:
sys_prompt = """You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into lines of code.
You should also analyze whether the variable in the optimization problem should be INTEGER or CONTINUOUS.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:\n"""

example = """
QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
VAR_TYPE:
INTEGER
CODE:
x = m.addVar(name="Oil Max", vtype=gp.GRB.INTEGER)
y = m.addVar(name="Oil Max Pro", vtype=gp.GRB.INTEGER)
m.setObjective(10 * x + 15 * y, gp.GRB.MAXIMIZE)
m.addConstr(46 * x + 13 * y <= 1345)
m.addConstr(43 * x + 4 * y <= 346)
m.addConstr(56 * x + 45 * y <= 1643)

QUESTION:
Ben is growing apples and pears on his orchard. He has 50 acres available on which he must grow a minimum of 5 acres of apples and a minimum of 10 acres of pears to meet demands. The profit per apple is $2 and the profit per pear is $4. He prefers to grow more pears than apples but limitations in his workforce allow him to grow at most twice the amount of pears as apples. How many of each fruit should Ben grow in order to maximize his profit? What is that profit?
VAR_TYPE:
INTEGER
CODE:
x = m.addVar(name="apples", vtype=gp.GRB.INTEGER)
y = m.addVar(name="pears", vtype=gp.GRB.INTEGER)
m.setObjective(2 * x + 4 * y, gp.GRB.MAXIMIZE)
m.addConstr(x + y <= 50)
m.addConstr(x >= 5)
m.addConstr(y >= 10)
m.addConstr(y <= 2 * x)
"""

prompt_template_str = sys_prompt + example + "\nPlease finish the task think step by step.\nQUESTION:{q}"
print(prompt_template_str)

You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into lines of code.
You should also analyze whether the variable in the optimization problem should be INTEGER or CONTINUOUS.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:

QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
VAR_

In [93]:
program = OpenAIPydanticProgram.from_defaults(
    output_cls=CodeScheme, 
    llm=OpenAI("gpt-3.5-turbo"),
    prompt_template_str=prompt_template_str.format(q=questions[2]), 
    verbose=True
)

In [94]:
code = program()

Function call: CodeScheme with args: {"var_type":"CONTINUOUS","code":"x = m.addVar(name=\"fertilizer A\", vtype=gp.GRB.CONTINUOUS)\ny = m.addVar(name=\"fertilizer B\", vtype=gp.GRB.CONTINUOUS)\nm.setObjective(5 * x + 9 * y, gp.GRB.MINIMIZE)\nm.addConstr(13 * x + 8 * y >= 220)\nm.addConstr(5 * x + 14 * y >= 160)\nm.addConstr(6 * x + 6 * y <= 350)"}


In [95]:
execute_code(complement_code(clean_code(code.code)))

125.49295774647888

In [98]:
# batch_size = 8
# lp_reasoning_list = []
# for idx in tqdm(range(0, len(questions), batch_size)):
#     batch = questions[idx:idx+batch_size]
    
#     tasks = [asyncclient.beta.chat.completions.parse(
#         model="gpt-4o-2024-08-06",
#         temperature=0,
#         response_format=Code,
#         messages=[
#             {"role": "system", "content": sys_prompt},
#             {"role": "user", "content": f"QUESTION: {q}"}
#         ]) for q in batch
#     ]

#     combined_responses = await asyncio.gather(*tasks)
#     lp_reasoning_list.extend([r.choices[0].message.parsed for r in combined_responses])

codes = []
for i in tqdm(range(len(questions))):
    program = OpenAIPydanticProgram.from_defaults(
        output_cls=CodeScheme, 
        llm=OpenAI("gpt-3.5-turbo"),
        prompt_template_str=prompt_template_str.format(q=questions[i]), 
        verbose=False
    )
    code = program()
    codes.append(code)

100%|██████████| 288/288 [10:39<00:00,  2.22s/it]


In [99]:
filename = 'e2e_codegen_gpt-3.5-turbo_lpwp_gurobi_' + dt + '.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
    pickle.dump(codes, f)

In [100]:
codes = [codes[i].code for i in range(len(codes))]

In [101]:
prefix = """
import gurobipy as gp
env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()
m = gp.Model(env=env)
"""
                
suffix = """
m.optimize()
"""

def complement_code(code: str) -> float:
    return prefix + code + suffix

In [102]:
def clean_code(code: str) -> str:
    cleand_code = []
    for line in code.split('\n'):
        line = line.strip()
        if line.startswith('m.addConstr') and not re.findall(r'<=|>=', line):
            line = re.sub(r'<', r'<=', line)
            line = re.sub(r'>', r'>=', line)
        cleand_code.append(line)
    cleand_code = '\n'.join(cleand_code)
    cleand_code = cleand_code.replace(')m', ')\nm')
    return cleand_code

In [103]:
def execute_code(code: str) -> float:
    ex_locals = {}
    exec(code, None, ex_locals)
    
    try:
        return ex_locals["m"].objVal
    except Exception as e:
        # print(e)
        return np.inf

In [104]:
pred_answers = []
for i, code_str in enumerate(codes):
    try:
        cleaned_code = clean_code(code_str)
        code = complement_code(cleaned_code)
        ans = execute_code(code)
        loguru.logger.info(f"question {i} obtain answer")
        pred_answers.append(ans)
    except Exception as e:
        loguru.logger.error(f"Error for question {i}: {e}")
        pred_answers.append("Error")

2024-09-18 20:22:41.255 | ERROR    | __main__:<module>:10 - Error for question 0: name 'cost_sled_dogs' is not defined
2024-09-18 20:22:41.256 | INFO     | __main__:<module>:7 - question 1 obtain answer
2024-09-18 20:22:41.258 | INFO     | __main__:<module>:7 - question 2 obtain answer
2024-09-18 20:22:41.260 | INFO     | __main__:<module>:7 - question 3 obtain answer
2024-09-18 20:22:41.261 | INFO     | __main__:<module>:7 - question 4 obtain answer
2024-09-18 20:22:41.262 | INFO     | __main__:<module>:7 - question 5 obtain answer
2024-09-18 20:22:41.265 | INFO     | __main__:<module>:7 - question 6 obtain answer
2024-09-18 20:22:41.266 | INFO     | __main__:<module>:7 - question 7 obtain answer
2024-09-18 20:22:41.267 | INFO     | __main__:<module>:7 - question 8 obtain answer
2024-09-18 20:22:41.269 | INFO     | __main__:<module>:7 - question 9 obtain answer
2024-09-18 20:22:41.272 | INFO     | __main__:<module>:7 - question 10 obtain answer
2024-09-18 20:22:41.274 | INFO     | __m

In [105]:
answers = [a if a != 'None' else str(np.inf) for a in answers]

In [106]:
def mark(pred, real, error: float) -> List[bool]:    
    correct = []
    for p, r in zip(pred, real):
        if p == 'Error':
            continue
        if float(r) != 0:
            if (float(p) == np.inf and float(r) == np.inf) or (abs(float(p) - float(r)) / float(r) < error):
                correct.append(True)
            else:
                correct.append(False)
        else:
            if float(p) < error:
                correct.append(True)
            else:
                correct.append(False)
    return correct

In [107]:
error1, error2 = 1e-2, 1e-4
print(f"Under {error1} error rate: the accuracy is {sum(mark(pred_answers, answers, error=error1)) / len(answers)}")
print(f"Under {error2} error rate: the accuracy is {sum(mark(pred_answers, answers, error=error2)) / len(answers)}")

Under 0.01 error rate: the accuracy is 0.6284722222222222
Under 0.0001 error rate: the accuracy is 0.6145833333333334
